# Joint definition of cell types from multiple scRNA-seq datasets (python version)

This notebook demonstrates the usage of the pyliger package.

In [1]:
# Please load following modules
import pyliger
from anndata import read_h5ad

/opt/anaconda3/lib/python3.7/site-packages/anndata/_core/anndata.py:21: FutureWarning: pandas.core.index is deprecated and will be removed in a future version.  The public classes are available in the top-level namespace.
  from pandas.core.index import RangeIndex


# Stage I: Preprocessing and Normalization (3 - 5 seconds)

0. (unfinished) Example of reading in 10X CellRanger output using read10X function. 

In [ ]:
sample_dirs = ['10x_ctrl_outs', '10x_stim_outs']
sample_names = ['ctrl', 'stim']

adata_list = read10X(sample_dirs, sample_names, merge = False)

1. For the first portion of this protocol, we will be integrating data from control and interferon-stimulated PBMCs from Kang et al, 2017. The data can be found in the Gene Expression Omnibus, Series GSE96583. This dataset was originally in the form of output from the 10X Cellranger pipeline, though we will directly load downsampled versions of the control and stimulated DGEs here.

    For convenience, we have prepared the pre-processed data which are ready to use. There are three datasets: “PBMC_control.h5ad.gz” and “PBMC_interferon-stimulated.h5ad.gz”, which correspond to control and interferon-stimulated PBMCs individually. 

In [2]:
ctrl_dge = read_h5ad('./pyliger/datasets/PBMC_control.h5ad')
stim_dge = read_h5ad('./pyliger/datasets/PBMC_interferon-stimulated.h5ad')

2. With the digital gene expression matrices for both datasets, we can initialize a pyliger object using createLiger function.

In [3]:
adata_list = [ctrl_dge, stim_dge]
ifnb_liger = pyliger.createLiger(adata_list)

Removing 20756 genes not expressing in ctrl.
Removing 21057 genes not expressing in stim.


3. Before we can run iNMF on our datasets, we must run several preprocessing steps to normalize expression data to account for differences in sequencing depth and efficiency between cells, identify variably expressed genes, and scale the data so that each gene has the same variance. Note that because nonnegative matrix factorization requires positive values, we do not center the data by subtracting the mean. We also do not log transform the data.

In [4]:
ifnb_liger = pyliger.normalize(ifnb_liger)
ifnb_liger = pyliger.selectGenes(ifnb_liger)
ifnb_liger = pyliger.scaleNotCenter(ifnb_liger)

4. 

# Stage II: Joint Matrix Factorization (3 - 10 minutes)

In [ ]:
#ifnb_liger = pyliger.optimizeALS(ifnb_liger, k = 20)

In [5]:
ifnb_liger = pyliger.iNMF_HALS(ifnb_liger, k = 20)

Initial Training Obj: 14433435.42277551
Iter: 1, Total time: 0.07457900047302246, Obj Delta: 1.9999999999994966
Iter: 2, Total time: 0.14094805717468262, Obj Delta: 1.2253752215814084
Iter: 3, Total time: 0.20684814453125, Obj Delta: 1.9652194739023687
Iter: 4, Total time: 0.2781031131744385, Obj Delta: 0.593886471810166
Iter: 5, Total time: 0.34319019317626953, Obj Delta: 1.287083274784265
Iter: 6, Total time: 0.4087991714477539, Obj Delta: 0.5614432894144349
Iter: 7, Total time: 0.4729270935058594, Obj Delta: 1.9999979329658466
Iter: 8, Total time: 0.5388922691345215, Obj Delta: 1.9999993378960228
Iter: 9, Total time: 0.6046593189239502, Obj Delta: 1.9999993148491748
Iter: 10, Total time: 0.669513463973999, Obj Delta: 1.9999981575684798
Iter: 11, Total time: 0.7365355491638184, Obj Delta: 1.9997605985511875
Iter: 12, Total time: 0.8074624538421631, Obj Delta: 1.6675891043716606
Iter: 13, Total time: 0.87497878074646, Obj Delta: 1.9802714047202257
Iter: 14, Total time: 0.9400479793548

5. 

# Stage III: Quantile Normalization and Joint Clustering (1 minute)

In [ ]:
ifnb_liger = quantile_norm(ifnb_liger)

6.

In [ ]:
ifnb_liger = louvainCluster(ifnb_liger, resolution = 0.25)

7. 

# Stage IV: Visualization (2 - 3 minutes) and Downstream Analysis (25 - 40 seconds)

In [ ]:
ifnb_liger = runUMAP(ifnb_liger, distance = 'cosine', n_neighbors = 30, min_dist = 0.3)

8.